In [1]:
# Le code commence par convertir les coordonnées au format décimal pour faciliter les calculs géographiques.
# Ensuite, les données de plusieurs fichiers sont chargées, les dates et heures sont extraites des noms de fichiers,
# et les itinéraires des bateaux sont organisés dans un ordre chronologique.
# Toutes les données sont combinées dans un tableau unique et triées par bateau et par heure.
# Une zone dangereuse autour des îles Canaries est définie sous forme de polygone,
# permettant de vérifier si des points ou des itinéraires de bateaux entrent ou traversent cette zone.
# Enfin, les bateaux dans la zone et les itinéraires traversant la zone sont affichés sur la carte.

In [2]:
import pandas as pd
import folium
from shapely.geometry import Point, LineString, Polygon

In [3]:
def convertir_en_decimal(coord):
    try:
        coord = coord.replace("'", "").strip()  
        degres, minutes = coord[:-1].split('°')  
        direction = coord[-1]  
        decimal = float(degres) + float(minutes) / 60
        if direction in ['S', 'W']:
            decimal = -decimal
        return decimal
    except Exception as e:
        print(f"Erreur")
        return None

In [4]:
fichiers = [
    'vendeeglobe_leaderboard_20241110_start.csv',
    'vendeeglobe_leaderboard_20241112_03.csv',
    'vendeeglobe_leaderboard_20241112_23.csv',
    'vendeeglobe_leaderboard_20241113_23.csv',
    'vendeeglobe_leaderboard_20241114_23.csv',
    'vendeeglobe_leaderboard_20241115_03.csv',
    'vendeeglobe_leaderboard_20241115_23.csv',
    'vendeeglobe_leaderboard_20241120_03.csv',
    'vendeeglobe_leaderboard_20241120_23.csv',
    'vendeeglobe_leaderboard_20241125_03.csv',
    'vendeeglobe_leaderboard_20241125_23.csv'
]

In [5]:
carte = folium.Map(location=[28.0, -15.0], zoom_start=6)

In [6]:
toutes_donnees = []

In [7]:
for fichier in fichiers:
    # Extraire la date du nom du fichier
    date = fichier.split('_')[2]  
    date = pd.to_datetime(date, format='%Y%m%d').strftime('%Y-%m-%d')  # format 'YYYY-MM-DD'

    # Déterminer l'heure à partir du nom du fichier
    if '03' in fichier:
        heure = '03:00'
    elif '23' in fichier:
        heure = '23:00'
    else:
        heure = '00:00'

    # Charger le fichier
    df = pd.read_csv(fichier, sep=';', encoding='latin1', header=0)
    df.columns = df.columns.str.strip()  # Nettoyer les noms des colonnes
    df.rename(columns={'Latitude Latitude': 'Latitude', 'Longitude Longitude': 'Longitude'}, inplace=True)

    df = df.dropna(subset=['Latitude', 'Longitude'])  # Supprimer les lignes avec des valeurs manquantes
    df['Latitude'] = df['Latitude'].astype(str).apply(convertir_en_decimal)  # Convertir la latitude
    df['Longitude'] = df['Longitude'].astype(str).apply(convertir_en_decimal)  # Convertir la longitude

    # Une colonne pour la date et l'heure complètes
    df['Date'] = date
    df['Heure'] = heure
    df['DateHeure'] = pd.to_datetime(df['Date'] + ' ' + df['Heure'])  # Combiner date et heure

    toutes_donnees.append(df)

# Combiner toutes les données
df_complet = pd.concat(toutes_donnees)

In [8]:
# Trier les données par bateau et par date/heure
df_complet = df_complet.sort_values(by=['Skipper / Bateau', 'DateHeure'])

In [9]:
# îles Canaries
coords_canaries = [
    [-18.2, 27.5],  # Coin inférieur gauche
    [-13.5, 27.5],  # Coin inférieur droit
    [-13.5, 29.5],  # Coin supérieur droit
    [-18.2, 29.5],  # Coin supérieur gauche
    [-18.2, 27.5]   # Retour au point initial
]
zone_canaries = Polygon(coords_canaries)

In [10]:
folium.Polygon(
    locations=[[lat, lon] for lon, lat in coords_canaries],
    color='blue',
    fill=True,
    fill_opacity=0.2,
    popup="Zone dangereuse - Îles Canaries"
).add_to(carte)

# Identifier les bateaux dans la zone et les routes qui traversent la zone
bateaux_zone = set()
routes_zone = set()

In [11]:
# Analyser les bateaux
for bateau in df_complet['Skipper / Bateau'].unique():
    donnees_bateau = df_complet[df_complet['Skipper / Bateau'] == bateau]

    # Vérifier les points de la route
    for _, ligne in donnees_bateau.iterrows():
        point = Point(ligne['Longitude'], ligne['Latitude'])
        if point.within(zone_canaries):
            bateaux_zone.add(bateau)

    # Vérifier les routes
    if len(donnees_bateau) >= 2:
        ligne_route = LineString(donnees_bateau[['Longitude', 'Latitude']].values)
        if ligne_route.intersects(zone_canaries):
            routes_zone.add(bateau)

In [12]:
# Ajouter les bateaux dans la zone sur la carte
for bateau in bateaux_zone:
    folium.Marker(
        location=[df_complet[df_complet['Skipper / Bateau'] == bateau]['Latitude'].iloc[0],
                 df_complet[df_complet['Skipper / Bateau'] == bateau]['Longitude'].iloc[0]],
        popup=f"Bateau {bateau} dans la zone",
        icon=folium.Icon(color='green')
    ).add_to(carte)

# Ajouter les routes traversant la zone
for bateau in routes_zone:
    donnees_bateau = df_complet[df_complet['Skipper / Bateau'] == bateau]
    coordonnees = donnees_bateau[['Latitude', 'Longitude']].values.tolist()
    folium.PolyLine(
        locations=coordonnees,
        color='red',
        weight=2.5,
        opacity=0.8,
        popup=f"Route du bateau {bateau} traversant la zone"
    ).add_to(carte)

folium.LayerControl().add_to(carte)

In [15]:
routes_dans_zone = []  # Создаём пустой список

# Добавляем точки только для маршрутов, пересекающих выбранную зону
for bateau in routes_dans_zone:
    # Фильтруем данные для текущей лодки
    donnees_bateau = df_complet[df_complet['Skipper / Bateau'] == bateau]

    # Проходим по каждой точке маршрута лодки
    for _, ligne in donnees_bateau.iterrows():
        folium.CircleMarker(
            location=[ligne['Latitude'], ligne['Longitude']],
            radius=4,
            color='orange',  # Цвет точек маршрута
            fill=True,
            fill_opacity=0.7,
            popup=f"Bateau: {bateau}<br>Date: {ligne['Date']}<br>Heure: {ligne['Heure']}"
        ).add_to(carte)


In [16]:
carte.save('analyse_zone_canaries.html')